In [3]:
# Import libraries
import os
import anthropic
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [13]:
# Loading environment variables
load_dotenv()
API_KEY = os.getenv("CLAUDE_API_KEY")
MODEL_NAME = os.getenv("CLAUDE_MODEL_NAME")

In [14]:
# Athropic client configuration
client = anthropic.Anthropic(api_key=API_KEY)

In [ ]:
# Loading file
def loading_document(path):
  
    with open(path, 'r', encoding='utf-8') as archivo:
        return archivo.read()

In [16]:
# Dividing document on chunks
def dividing_document(text, chunk_size=1000, chunk_overlap=200):

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

In [17]:
# Creation embeddings for chunnks
def creation_embeddings(chunks):
    
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
    vectorstore = FAISS.from_texts(chunks, embedding=embeddings)
    return vectorstore


In [18]:
# Recovering the most important chunks for the consult
def recovering_context(vectorstore, query, top_k=3):
    
    return vectorstore.similarity_search(query, k=top_k)

In [19]:
# Generate an aswer usig Claude with the recovered context
def generate_answer_with_context(query, context):
    context_text = "\n".join([doc.page_content for doc in context])
    
    system_prompt = """Eres un asistente experto en análisis de documentos técnicos.
    Utiliza el contexto proporcionado para responder de manera precisa y detallada.
    Si la información no está en el contexto, indica que no puedes responder."""
    
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
            {"role": "user", "content": f"Contexto: {context_text}\n\nPregunta: {query}"}
        ]
    )
    
    return message.content[0].text

In [20]:
# Reflection process for improving the answer
def reflection_process(pregunta, initial_answer):

    system_prompt = "Eres un analista técnico crítico que evalúa soluciones de manera objetiva."
    
    # Evaluation
    evaluation = client.messages.create(
        model=MODEL_NAME,
        system=system_prompt,
        max_tokens=2000,
        messages=[
            {"role": "user", "content": f"Evalúa críticamente esta respuesta:\n{initial_answer}\n\nCriterios de evaluación:\n- Precisión\n- Completitud\n- Claridad\n- Relevancia"}
        ]
    ).content[0].text
    
    # Auto-reflection
    autoreflection = client.messages.create(
        model=MODEL_NAME,
        system=system_prompt,
        max_tokens=2000,
        messages=[
            {"role": "user", "content": f"Basándote en la respuesta y evaluación:\n\nRespuesta: {initial_answer}\n\nEvaluación: {evaluation}\n\nGenera una reflexión con estrategias de mejora."}
        ]
    ).content[0].text
    
    # Better answer
    better_answer = client.messages.create(
        model=MODEL_NAME,
        system=system_prompt,
        max_tokens=2000,
        messages=[
            {"role": "user", "content": f"Partiendo de:\nPregunta Original: {pregunta}\nRespuesta Inicial: {initial_answer}\nEvaluación: {evaluation}\nReflexión: {autoreflection}\n\nGenera una versión MEJORADA de la respuesta."}
        ]
    ).content[0].text
    
    return {
        'respuesta_inicial': initial_answer,
        'evaluacion': evaluation,
        'autoreflexion': autoreflection,
        'respuesta_mejorada': better_answer
    }


In [21]:
# Consultig Documentation
def consulting_doc(query):
   
    # Loading and processing document
    document = loading_document("./info.txt")
    chunks = dividing_document(document)
    vectorstore = creation_embeddings(chunks)
    
    # Recovering important context
    context = recovering_context(vectorstore, query)
    
    # Generation of the initial answer with RAG
    initial_answer = generate_answer_with_context(query, context)
    
    # Reflection process
    resultado_reflexion = reflection_process(query, initial_answer)
    
    return resultado_reflexion

In [22]:
# Use examples
query = "¿Cuáles son los objetivos principales del proyecto Nebula?"
result = consulting_doc(query)

print("### Respuesta Inicial ###")
print(result['respuesta_inicial'])
print("\n### Evaluación ###")
print(result['evaluacion'])
print("\n### Auto-Reflexión ###")
print(result['autoreflexion'])
print("\n### Respuesta Mejorada ###")
print(result['respuesta_mejorada'])

### Respuesta Inicial ###
De acuerdo con el contexto proporcionado, los principales objetivos del proyecto Nebula son:

1. Proporcionar un sistema SaaS accesible y eficiente para la creación y evaluación de modelos de aprendizaje automático, específicamente para tareas de regresión y clasificación.

2. Diseñar una plataforma que brinde una experiencia intuitiva y automatizada, permitiendo a usuarios con diversos niveles de experiencia en ciencia de datos crear modelos precisos sin necesidad de conocimientos avanzados en machine learning.

3. Lograr que la arquitectura de Nebula sea escalable y modular, facilitando la integración de nuevas funcionalidades y mejoras a medida que evolucionen las necesidades de los usuarios.

En resumen, los objetivos principales son proporcionar una plataforma SaaS accesible y eficiente para el desarrollo de modelos de aprendizaje automático, con una arquitectura escalable y modular que permita adaptarse a las necesidades cambiantes de los usuarios.

### 